In [ ]:
import numpy as np
import util

from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten

In [ ]:
dataset = np.load('./datasets/v0_eigens.npz')

train_data_size = dataset['train_eigens'].shape[0]
valid_data_size = train_data_size / 5
train_data_size = train_data_size - valid_data_size

indices = np.arange(train_data_size + valid_data_size)

train_data = dataset['train_eigens'][indices[:train_data_size]]
valid_data = dataset['train_eigens'][indices[train_data_size:]]

train_eigens = train_data[:, :-28].reshape(-1, 32, 28, 1)
train_labels = train_data[:, -28:]
valid_eigens = valid_data[:, :-28].reshape(-1, 32, 28, 1)
valid_labels = valid_data[:, -28:]
issue_eigens = dataset['issue_eigens'][:, :-28].reshape(-1, 32, 28, 1)

print 'train_eigens.shape = {}'.format(train_eigens.shape)
print 'train_labels.shape = {}'.format(train_labels.shape)
print 'valid_eigens.shape = {}'.format(valid_eigens.shape)
print 'valid_labels.shape = {}'.format(valid_labels.shape)

In [ ]:
def build_model():
    """
    """
    model = Sequential()

    model.add(Conv2D(8, (5, 5), activation='relu', input_shape=(32, 28, 1)))
    model.add(Conv2D(8, (5, 5), activation='relu'))
    model.add(Conv2D(8, (5, 5), activation='relu'))
    model.add(Conv2D(8, (5, 5), activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(28, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy')
    
    return model

In [ ]:
model = build_model()

model.fit(
    x=train_eigens,
    y=train_labels,
    batch_size=128,
    epochs=5,
    verbose=1,
    validation_data=(valid_eigens, valid_labels),
    shuffle=True)

known best: 0.880813190441

In [ ]:
valid_guesss = model.predict(valid_eigens)

auc = util.auc(valid_guesss, valid_labels)

print auc

In [ ]:
issue_guesss = model.predict(issue_eigens)

util.write_result('cnn.csv', issue_guesss)